In [ ]:
using Revise
push!(LOAD_PATH, "../src")
using AllenCahn

In [ ]:
left_bc = NeumannBC(0)
right_bc = NeumannBC(0)
problem = AllenCahnProblem1D(0, 1, 0.01, 10, 5, left_bc, right_bc);

In [ ]:
println("a: $(problem.a), b: $(problem.b), ϵ: $(problem.ϵ), Δt: $(problem.Δt)")
println("mesh: $(problem.x)")

In [ ]:
assemble_system!(problem)